### Concept of the app
1. Get Articles (once, weekly, daily, ...)

    a. scraping & parsing
    
    b. API access

    c. save to DB!!!

2. Recognise keywords and main topic
    
    a. TF-IDF
    
    b. ???

3. Group with other of the same topic
    
    a. Main topic matching
    
    b. Hierarchy (decision tree?)

4. Find divergences

    a. diff on keyword set

    b. difference of opinion

    c. difference of data sources

    d. difference in 

5. Represent articles and divergences

In [ ]:
import os
import sys
import time
import lxml
import json
import shutil
import codecs
import requests
import psycopg2
import sqlalchemy
import urllib.parse

import numpy as np
import pandas as pd

from random import random
from datetime import datetime
from datetime import timezone
from selenium.webdriver import Chrome

sys.path.append("../")

import utils.NYTParser
from utils.sql_utils import *
from utils.scrape_utils import *

In [ ]:
pipelineNYT(name='nyt_20200723_1600', fetchSource=True)

driver.get("https://www.nytimes.com/")

Possible sources:

    "NYTimes": "https://www.nytimes.com/",
    "BBC": "https://www.bbc.co.uk/",
    "CNN": "https://edition.cnn.com/",
    "FoxNews": "https://www.foxnews.com/",
    "OAN": "https://www.oann.com/"

The Guardian, Yahoo news, Washington Post, Daily Mail, ...

https://www.4imn.com/top200/ for more

In [ ]:
def saveData(data, name):
    with open('../raw_data/%s.txt' % name, 'w') as outfile:
        json.dump(data, outfile)

In [ ]:
def loadData(name):
    with open('../raw_data/%s.txt' % name) as json_file:
        data = json.load(json_file)
    return data

In [ ]:
articles = fetchNewsAPI(content_type="articles", url=('http://newsapi.org/v2/top-headlines?'
        'pageSize=50&'
        'page={page}&'
        'language=en&'
        'country=us&'
        'apiKey=e30a64cfe1734e6794bdab67106590fa'))
len(articles)

In [ ]:
articles[0].keys()

In [ ]:
articles[0]['source']['name']

In [ ]:
provider_uuid = getSourceID('NewsAPI')

In [ ]:
for a in articles:
    print(a['source']['name'])
    source_uuid = getSourceID(a['source']['name'])
    if source_uuid is not None:
        print(source_uuid)
        a['article_url'] = a['url']
        a['source_uuid'] = source_uuid
        a['provider_uuid'] = provider_uuid
        insertArticle(a)
    else:
        print("Could not match source:", a['source']['name'])
        print(a)

In [ ]:
a = getSourceID('The Wall Street Journal')

In [ ]:
getSourceID('New York Post')

In [ ]:
def insertArticle(params):

    possible_article_params = ['article_url',
                               'source_uuid',
                               'provider_uuid',
                               'title',
                               'description',
                               'author',
                               'publishedAt',
                               'updatedAt']

    engine = getEngine()
    params = {k: v for k, v in params.items() if k in possible_article_params}
    sql = f"""insert into media.listing.articles ({", ".join(params.keys())})
            values ({list_for_sql(params.values())});"""
    try:
        engine.execute(sql)
        if 'article_url' in params and 'description' in params:
            article_uuid_sql = f"""select article_uuid from articles where article_url = '{params['article_url']}';"""
            article_uuid = str(engine.execute(
                article_uuid_sql).fetchall()[0][0])
            description_sql = f"""insert into media.listing.article_contents
                values ('{article_uuid}', '{params['description']}');"""
            engine.execute(description_sql)
    except (psycopg2.errors.UniqueViolation, sqlalchemy.exc.IntegrityError):
        if 'article_url' in params:
            print("Article article already exists:", params['article_url'])
        else:
            print("Article url is required.")
        # todo manage article duplicates with difference or different sources

In [ ]:
"abca".replace("a", "b")

In [ ]:
pipelineNYT(name='nyt_20200722_1000', fetchSource=True)

In [ ]:
test_article_url = "https://www.nytimes.com/2020/07/21/business/economy/coronavirus-unemployment-benefits.html"
test_article_uri = "nyt://article/6fae2333-ff5a-5ac0-bcc5-34a2da53cbb3"